## Importing Libraries 

In [100]:
import pandas as pd 
import nltk 
import re 
from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize


## Downlaoding NLTK packages

In [101]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bilal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bilal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Loading and preprocessing the Data

In [102]:
df_true = pd.read_csv('../Data/True.csv')
df_fake = pd.read_csv('../Data/Fake.csv')

df_true['label'] = 0
df_fake['label'] = 1

df_true


,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


In [103]:
df_fake

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


## Concaat + Randomise the Data

In [104]:
df = pd.concat([df_true, df_fake], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

### Dropping the date and subject column as it doesn't contribute to the models learning

In [105]:
df = df.drop(columns=['date','subject'], axis=1)

In [106]:
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

df['content'] = df['title'] + ' ' + df['text']

df = df[['content', 'label']]

In [107]:
df

,content,label
0,OBAMA’S EMBARRASSING Farewell Interview: Mom W...,1
1,"China's Guangdong gets new party boss, former ...",0
2,Pentagon notified U.S. Congress of $593 millio...,0
3,Belgian soldiers shoot dead knife attacker in ...,0
4,Thousands of Romanians rally against ruling pa...,0
...,...,...
44893,PARENT FURIOUS AFTER 4TH GRADE CLASS JOINS HAT...,1
44894,Trump administration proposes stricter Obamaca...,0
44895,‘Feel Good Hoax’: How US Propaganda Works in t...,1
44896,'Unremarkable' Virginia attacker shows difficu...,0


### Exploring The Data

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  44898 non-null  object
 1   label    44898 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 701.7+ KB


In [109]:
df.describe()

,label
count,44898.000000
mean,0.522985
std,0.499477
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [110]:
df.isna().sum()

content    0
label      0
dtype: int64

In [111]:
df.isnull().sum()

content    0
label      0
dtype: int64

In [112]:
stop__words = set(stopwords.words('english'))

### Cleaning of Data 

In [ ]:
def cleanText(text):
    text = str(text).lower()
    text = re.sub(r'\d+','', text)
    text =  re.sub(r'[^\w\s]', '', text) 
    words = text.split()    
    words = [word for word in words if word not in stop__words]
    return " ".join(words)


### Applying the clean function to the Content Columns

In [118]:

df['content'] = df['content'].apply(cleanText)


df

,content,label
0,obamas embarrassing farewell interview mom hip...,1
1,chinas guangdong gets new party boss former le...,0
2,pentagon notified us congress million military...,0
3,belgian soldiers shoot dead knife attacker bru...,0
4,thousands romanians rally ruling partys judici...,0
...,...,...
44893,parent furious th grade class joins hateful pr...,1
44894,trump administration proposes stricter obamaca...,0
44895,feel good hoax us propaganda works syrian war ...,1
44896,unremarkable virginia attacker shows difficult...,0


### Exporting the Data

In [117]:
df.to_csv("CleanedData")